# Agentic Evaluation (ReAct-style)

This notebook adds a minimal ReAct-style loop for NL→SQL. It reuses the same benchmark (`data/classicmodels_test_200.json`) and metrics (VA/EX/EM; TS planned) to measure gains over prompt-only and QLoRA runs.

Plan (step-by-step):
1) Clone repo (Colab) + install deps
2) Environment + DB connection
3) Load schema summary + test set
4) Load model (base or QLoRA adapters)
5) Define ReAct prompt + loop (Thought → Action → Observation → Refinement)
6) Run evaluation (VA/EX/EM) and save to `results/agent/…`


Docs I leaned on: HF Transformers quantization (https://huggingface.co/docs/transformers/main_classes/quantization), PEFT/TRL (https://huggingface.co/docs/peft/, https://huggingface.co/docs/trl/), Cloud SQL connector + SQLAlchemy creator (https://cloud.google.com/sql/docs/mysql/connect-run, https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect), ReAct (https://arxiv.org/abs/2210.03629).

## Setup (run first, then restart)
In a fresh Colab GPU runtime, run this one cell to clean preinstalls and pin the CUDA 12.1 torch/bitsandbytes/triton stack. When it finishes, **Runtime → Restart runtime**, then run the rest of the notebook from the clone cell onward without more restarts.

**Docs (setup):** HF Transformers quantization + BitsAndBytes (4-bit) https://huggingface.co/docs/transformers/main_classes/quantization, bnb https://github.com/TimDettmers/bitsandbytes.

In [ ]:

%%bash
set -e
export PIP_DEFAULT_TIMEOUT=120

# Clean conflicting preinstalls
pip uninstall -y torch torchvision torchaudio bitsandbytes triton transformers accelerate peft trl datasets numpy pandas fsspec requests google-auth || true

# Base deps
pip install -q --no-cache-dir --force-reinstall   numpy==1.26.4 pandas==2.2.1 fsspec==2024.5.0 requests==2.31.0 google-auth==2.43.0

# Torch + CUDA 12.1
pip install -q --no-cache-dir --force-reinstall   torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121   --index-url https://download.pytorch.org/whl/cu121

# bitsandbytes + triton + HF stack
pip install -q --no-cache-dir --force-reinstall   bitsandbytes==0.43.3 triton==2.3.1   transformers==4.44.2 accelerate==0.33.0 peft==0.17.0 trl==0.9.6 datasets==2.20.0

echo "Setup complete. Restart runtime once, then run the rest of the notebook top-to-bottom."


Model load: HF 4-bit NF4 + BitsAndBytes; deterministic decoding. If adapters exist, we load them.

In [ ]:
# 0) Clone repo (Colab) + install deps
import os
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    if not os.path.exists('/content/NLtoSQL'):
        !git clone https://github.com/MacKenzieOBrian/NLtoSQL.git /content/NLtoSQL
    %cd /content/NLtoSQL
    !pip -q install -r requirements.txt
    import torch, transformers, accelerate, peft
    print('torch', torch.__version__, 'cuda', torch.cuda.is_available())
else:
    print('Not in Colab; using existing workspace')


Prompt/eval: build prompts (system+schema+k exemplars), generate SQL, postprocess, and compute VA/EX/EM.

**Ref:** Colab clone/install pattern; keeps notebooks thin and code in `nl2sql/`. Hugging Face/Colab standard workflow.

### Reference notes (what this builds on)
- DB access: Cloud SQL Connector + SQLAlchemy creator (GCP docs: https://cloud.google.com/sql/docs/mysql/connect-run) for secure pooled ClassicModels access.
- Schema/prompting: uses repo helpers (`nl2sql.schema`, `prompting`) aligned with schema-grounded NL→SQL prompting (survey: https://arxiv.org/abs/2410.06011).
- Model load: HF Transformers 4-bit NF4 with BitsAndBytes (quantization docs: https://huggingface.co/docs/transformers/main_classes/quantization), same pattern as QLoRA.
- Agent loop: ReAct-style Thought→Action→Observation→Refinement, inspired by Yao et al. 2023 (https://arxiv.org/abs/2210.03629) and agentic NL→SQL in Ojuri et al. 2025.
- Eval: repo harness (`nl2sql.eval`, `QueryRunner`) for VA/EX/EM; TS planned.


## Optional: use gcloud ADC (without a key)

**Ref:** GCP ADC flow (docs: https://cloud.google.com/docs/authentication/provide-credentials-adc). Optional fallback if no service account JSON.

In [ ]:
# Run this only if you prefer gcloud-based ADC (no JSON key)
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    %pip install -q --upgrade google-auth google-auth-oauthlib
    !gcloud auth application-default login
else:
    print("Not in Colab; skip gcloud auth.")


**Ref:** Pinned CUDA12.1 torch/bitsandbytes/triton stack per HF/BnB guidance for 4-bit loads on Colab GPUs.

**Ref:** Cloud SQL Connector + SQLAlchemy creator (GCP MySQL docs: https://cloud.google.com/sql/docs/mysql/connect-run) for secure ClassicModels access.

**Docs (auth/DB):** Cloud SQL connector pattern https://cloud.google.com/sql/docs/mysql/connect-run; SQLAlchemy creator hook https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect.

In [ ]:

# 1) Environment + DB
import os
from getpass import getpass
from pathlib import Path

from google.cloud.sql.connector import Connector
from google.oauth2.service_account import Credentials
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine

# Expected env vars (set these in a Colab cell):
# GOOGLE_APPLICATION_CREDENTIALS=/content/sa.json
# INSTANCE_CONNECTION_NAME, DB_USER, DB_PASS, DB_NAME
INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME") or "classicmodels"
GOOGLE_CREDS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

if not INSTANCE_CONNECTION_NAME:
    INSTANCE_CONNECTION_NAME = input("Enter INSTANCE_CONNECTION_NAME: ").strip()
if not DB_USER:
    DB_USER = input("Enter DB_USER: ").strip()
if not DB_PASS:
    DB_PASS = getpass("Enter DB_PASS: ")

creds = None
if GOOGLE_CREDS:
    creds = Credentials.from_service_account_file(GOOGLE_CREDS)
    print(f"Using service account from {GOOGLE_CREDS}")
else:
    print("Using default ADC (gcloud auth or Colab auth). If this fails, set GOOGLE_APPLICATION_CREDENTIALS.")

connector = Connector(credentials=creds)

def getconn():
    return connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,
    )

engine: Engine = create_engine(
    "mysql+pymysql://",
    creator=getconn,
    future=True,
)

with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print("DB connection OK")


**Ref:** Schema helper in `nl2sql.schema`; schema-grounded prompting per NL→SQL survey (https://arxiv.org/abs/2410.06011).

**Docs (schema prompts):** NL→SQL schema-grounded prompting survey https://arxiv.org/abs/2410.06011; Spider-style listings.

In [ ]:
# 2) Load schema summary + test set (small slice for now)
import json
from nl2sql.schema import build_schema_summary

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME)

test_path = Path("data/classicmodels_test_200.json")
full_set = json.loads(test_path.read_text(encoding="utf-8"))
# default to a small slice while debugging
test_set = full_set[:5]
print("Demo items:", len(test_set))
# For full run, switch to: test_set = full_set; print("Test items:", len(test_set))


**Ref:** HF Transformers 4-bit NF4 + BitsAndBytes (quantization docs: https://huggingface.co/docs/transformers/main_classes/quantization); adapters via PEFT.

**Docs (model load):** HF 4-bit NF4 quantization https://huggingface.co/docs/transformers/main_classes/quantization; PEFT/QLoRA https://huggingface.co/docs/peft/.

In [ ]:

# 3) Load model (base or QLoRA adapters)
import os
from getpass import getpass
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
ADAPTER_PATH = os.getenv("ADAPTER_PATH") or "results/adapters/qlora_classicmodels"  # set to None to use base model

HF_TOKEN = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_TOKEN")
if not HF_TOKEN:
    HF_TOKEN = getpass("Enter HF_TOKEN (https://huggingface.co/settings/tokens): ").strip()

cc_major, cc_minor = torch.cuda.get_device_capability(0) if torch.cuda.is_available() else (0, 0)
use_bf16 = cc_major >= 8
compute_dtype = torch.bfloat16 if use_bf16 else torch.float16
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
print("Using bf16:", use_bf16)
print("Adapter path:", ADAPTER_PATH)

# Tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Quantized base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    device_map={"": 0} if torch.cuda.is_available() else None,
    token=HF_TOKEN,
)
base_model.generation_config.do_sample = False
base_model.generation_config.temperature = 1.0
base_model.generation_config.top_p = 1.0

# Load adapters if present locally; otherwise use base model
adapter_dir = Path(ADAPTER_PATH) if ADAPTER_PATH else None
if adapter_dir and adapter_dir.exists():
    model = PeftModel.from_pretrained(base_model, adapter_dir, token=HF_TOKEN)
    print("Loaded adapters from", adapter_dir)
else:
    print("Adapter path missing; using base model only. Set ADAPTER_PATH to your local adapter folder or upload it to Colab.")
    model = base_model


## Optional adapter sanity check (run before ReAct)
Quick check to see if the loaded model/adapters produce valid SQL on a tiny slice. Uses the prompt harness (k=0/k=3) and executes the SQL to report VA/EX.

**Docs (prompt/eval):** ICL patterns https://arxiv.org/abs/2005.14165; execution-based metrics (VA/EX) https://aclanthology.org/2020.emnlp-main.29/.

In [ ]:
from nl2sql.prompting import make_few_shot_messages
from nl2sql.llm import extract_first_select
from nl2sql.postprocess import guarded_postprocess
from nl2sql.query_runner import QueryRunner
from nl2sql.eval import execution_accuracy

runner_check = QueryRunner(engine)
# reuse existing test_set (default small slice); pick 3 exemplars
exemplars = test_set[:3]

def run_quick_check(k: int = 0, limit: int = 3):
    print(f"Quick check k={k}")
    for sample in test_set[:limit]:
        shots = exemplars if k > 0 else []
        msgs = make_few_shot_messages(
            schema=SCHEMA_SUMMARY,
            exemplars=shots,
            nlq=sample['nlq'],
        )
        prompt_preview = tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
        inputs = tok(prompt_preview, return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=256, do_sample=False)
        text = tok.decode(out[0], skip_special_tokens=True)

        raw_sql = extract_first_select(text) or text
        sql = guarded_postprocess(raw_sql, sample['nlq'])

        meta = runner_check.run(sql, capture_df=False)
        va = meta.success
        ex, _, _ = execution_accuracy(engine=engine, pred_sql=sql, gold_sql=sample['sql'])
        print(f"Q: {sample['nlq']}
SQL: {sql}
VA: {va} EX: {ex}
")

run_quick_check(k=0)
run_quick_check(k=3)


**Ref:** ReAct pattern (Yao et al. 2023: https://arxiv.org/abs/2210.03629) adapted for NL→SQL with `QueryRunner` as the Act step.

**Docs (ReAct):** ReAct loop (Yao et al. 2023) https://arxiv.org/abs/2210.03629; safe Act via SELECT-only executor.

In [ ]:
# Helper: clamp ORDER BY if not requested, trim extra columns for simple list/which queries
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML

def _has_keyword(text, keywords):
    lt = text.lower()
    return any(k in lt for k in keywords)

def strip_order_by_if_not_requested(sql: str, nlq: str) -> str:
    if _has_keyword(nlq, ["order", "sort", "top", "first", "highest", "lowest", "descending", "ascending", "limit"]):
        return sql
    parsed = sqlparse.parse(sql)
    if not parsed:
        return sql
    stmt = parsed[0]
    tokens = []
    skip = False
    for tok in stmt.tokens:
        if tok.ttype is Keyword and tok.value.upper() == 'ORDER BY':
            skip = True
        if not skip:
            tokens.append(tok)
    cleaned = ''.join(str(t) for t in tokens).strip()
    return cleaned if cleaned.endswith(';') else cleaned + ';'

def trim_to_first_column(sql: str) -> str:
    parsed = sqlparse.parse(sql)
    if not parsed:
        return sql
    stmt = parsed[0]
    new_tokens = []
    seen_select = False
    trimmed = False
    for tok in stmt.tokens:
        if tok.ttype is DML and tok.value.upper() == 'SELECT':
            seen_select = True
            new_tokens.append(tok)
            continue
        if seen_select and isinstance(tok, IdentifierList) and not trimmed:
            first = [tok.get_identifiers().__next__()]
            new_tokens.append(IdentifierList(first))
            trimmed = True
            continue
        if seen_select and isinstance(tok, Identifier) and not trimmed:
            new_tokens.append(tok)
            trimmed = True
            continue
        new_tokens.append(tok)
    cleaned = ''.join(str(t) for t in new_tokens).strip()
    return cleaned if cleaned.endswith(';') else cleaned + ';'

def apply_clamps(sql: str, nlq: str) -> str:
    sql = strip_order_by_if_not_requested(sql, nlq)
    if _has_keyword(nlq, ["which", "who", "what", "list", "show"]) and ' and ' not in nlq.lower():
        # heuristically keep only first projection for simple list/which questions
        sql = trim_to_first_column(sql)
    return sql


In [ ]:
# Helper: generate and rerank multiple SQL candidates
import torch
import transformers
from nl2sql.llm import extract_first_select
from nl2sql.postprocess import guarded_postprocess

# Quiet HF warning spam and ensure pad token set once
transformers.logging.set_verbosity_error()
if tok.pad_token is None:
    tok.pad_token = tok.eos_token


def build_react_prompt(nlq: str, schema_text: str, history: list[dict], observation: str) -> str:
    history_text = "
".join([
        f"Thought/Action: {h.get('ta','')}
Observation: {h.get('obs','')}" for h in history
    ])
    return f"""
You are an expert MySQL analyst. Given a question and schema, propose ONE SELECT query only.
Rules: single SELECT; no DDL/DML/comments; use only schema columns; no ORDER/LIMIT unless question asks for ranking; minimal columns; join customers for country/creditLimit; join products (+orderdetails for totals) for productLine/vendor; totals use SUM(orderdetails.quantityOrdered * orderdetails.priceEach) grouped by orderNumber; status literals are Shipped, Cancelled, On Hold, Disputed, In Process, Resolved.
Schema:
{schema_text}

Question: {nlq}
Recent steps:
{history_text}
Last observation: {observation}
Return only the SQL statement starting with SELECT.
"""


def projection_guard(sql: str, nlq: str) -> str:
    return guarded_postprocess(sql, nlq)


def apply_clamps(sql: str, nlq: str) -> str:
    return sql


def generate_candidates(prompt: str, num: int = 2):
    inputs = tok(prompt, return_tensors="pt", padding=True).to(model.device)
    cands = []
    with torch.no_grad():
        for _ in range(num):
            out = model.generate(
                **inputs,
                max_new_tokens=192,
                do_sample=False,
            )
            gen_ids = out[0][inputs.input_ids.shape[-1]:]
            gen = tok.decode(gen_ids, skip_special_tokens=True)
            cands.append(gen)
    return cands


def count_select_columns(sql: str) -> int:
    lower = sql.lower()
    if ' from ' not in lower:
        return 99
    select_part = lower.split(' from ')[0]
    if 'select' in select_part:
        select_part = select_part.split('select',1)[1]
    return select_part.count(',') + 1


def react_sql(nlq: str, schema_text: str, max_steps: int = 3, num_cands: int = 2):
    history = []
    observation = "Start."
    final_sql = None

    for step in range(max_steps):
        prompt = build_react_prompt(nlq, schema_text, history, observation)
        raw_cands = generate_candidates(prompt, num=num_cands)
        ranked = []
        for raw in raw_cands:
            sql = extract_first_select(raw) or raw
            sql = sql if sql.endswith(';') else sql + ';'
            sql = projection_guard(sql, nlq)
            sql = apply_clamps(sql, nlq)
            ranked.append(sql)
        best_sql = ranked[0]
        for sql in ranked:
            try:
                meta = runner.run(sql)
                if meta.success:
                    best_sql = sql
                    break
            except Exception:
                continue
        try:
            meta = runner.run(best_sql)
            if meta.success:
                observation = "SUCCESS"
                final_sql = best_sql
                break
            else:
                observation = f"ERROR: {meta.error or 'execution failed'}"
        except Exception as e:
            observation = f"ERROR: {e}"
        history.append({"ta": best_sql, "obs": observation})

    return final_sql or best_sql, history


**Ref:** Repo eval (`nl2sql.eval`) for VA/EX/EM; execution-based metrics align with Ojuri et al. 2025 and EMNLP’20 TS.

**Docs (prompt/eval):** ICL patterns https://arxiv.org/abs/2005.14165; execution-based metrics (VA/EX) https://aclanthology.org/2020.emnlp-main.29/.

In [ ]:
# 5) Evaluation loop (VA/EX/EM). TS is planned.
from nl2sql.eval import execution_accuracy

results = []
for i, sample in enumerate(test_set, start=1):
    nlq = sample["nlq"]
    gold_sql = sample["sql"]
    pred_sql, trace = react_sql(nlq, SCHEMA_SUMMARY, max_steps=3)
    va = 0
    em = int(pred_sql.strip().rstrip(";").lower() == gold_sql.strip().rstrip(";").lower())
    ex = 0
    try:
        runner.run(pred_sql)
        va = 1
        ex = int(execution_accuracy(engine, pred_sql, gold_sql))
    except Exception:
        va = 0
        ex = 0
    results.append({
        "nlq": nlq,
        "gold_sql": gold_sql,
        "pred_sql": pred_sql,
        "va": va,
        "em": em,
        "ex": ex,
        "trace": trace,
    })
    if i % 20 == 0:
        print(f"Processed {i}/{len(test_set)}")

va_rate = sum(r["va"] for r in results) / len(results)
ex_rate = sum(r["ex"] for r in results) / len(results)
em_rate = sum(r["em"] for r in results) / len(results)
print("VA:", va_rate, "EX:", ex_rate, "EM:", em_rate)

Path("results/agent").mkdir(parents=True, exist_ok=True)
save_path = Path("results/agent/results_react_200.json")
save_path.write_text(json.dumps({
    "va_rate": va_rate,
    "ex_rate": ex_rate,
    "em_rate": em_rate,
    "items": results,
}, ensure_ascii=False, indent=2), encoding="utf-8")
print("Saved to", save_path)
